
# Assignment 3: Gradebook & Log Analyzer

**Covers:** Conditionals & Loops • File I/O • Dictionaries, Tuples & Methods  
**Estimated time:** ~2–3 hours

## Learning outcomes
By completing this assignment, you will be able to:
1. Use conditionals and loops to implement program logic.
2. Read from and write to text/CSV files safely.
3. Model data with dictionaries and tuples; use built-in methods effectively.
4. Build a small, menu-driven CLI program that ties everything together.



## Instructions
- Work in this notebook. Replace each `# TODO` with your code.
- Do **not** change the function names or signatures unless asked.
- Run the provided tests (asserts) to self-check. Passing tests ≠ perfect score.
- When you finish, **restart & run all** to ensure it executes cleanly top-to-bottom.
- Submit the exported `.ipynb` (and any generated `.txt`/`.csv` files).

In [ ]:

# --- Setup: create tiny sample datasets (run once) ---
from pathlib import Path
import csv

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# Sample scores.csv: student_id,name,quiz1,quiz2,quiz3
scores_path = data_dir / "scores.csv"
if not scores_path.exists():
    with scores_path.open("w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["student_id", "name", "quiz1", "quiz2", "quiz3"])
        writer.writerows([
            ["S001", "Aye", 8, 9, 10],
            ["S002", "Khin", 7, 6, 9],
            ["S003", "Ryan", 10, 10, 9],
            ["S004", "Hugo", 5, 7, 6],
            ["S005", "Thiri", 9, 8, 8],
        ])

# Sample access.log: username,action,timestamp (ISO-like, simplified)
log_path = data_dir / "access.log"
if not log_path.exists():
    log_path.write_text(
        "aye,login,2025-08-01T09:00\n"
        "aye,submit,2025-08-01T09:10\n"
        "khin,login,2025-08-01T09:05\n"
        "ryan,login,2025-08-01T09:07\n"
        "ryan,download,2025-08-01T09:20\n"
        "ryan,submit,2025-08-01T09:35\n"
        "hugo,login,2025-08-01T09:40\n"
        "thiri,login,2025-08-01T09:41\n"
        "aye,download,2025-08-01T09:50\n"
    )
print("Sample data ready at:", data_dir.resolve())



---
## Part A — Warm‑ups (Conditionals, Loops, Tuples)

### A1. `letter_grade(avg)`
Write a function that converts a numeric average (0–10) to a letter grade:
- `>= 9.0` → **A**
- `>= 8.0` → **B**
- `>= 7.0` → **C**
- `>= 6.0` → **D**
- else → **F**

Edge cases: floor to one decimal place before comparing (e.g., `8.96 → 8.9`).

### A2. `count_vowels(s)`
Return a dictionary mapping vowel → count for the string `s`. Treat `'aeiou'` as vowels and ignore case.

### A3. `rotate_tuple(t)`
Given a non-empty tuple `t`, return a **new** tuple where the last element moves to the front. Example: `(1,2,3) → (3,1,2)`.


In [7]:
# TODO: A1
def letter_grade(avg: float) -> str:
    # Floor to one decimal place
    floored = float(f"{avg:.1f}")
    if floored >= 9.0:
        return "A"
    elif floored >= 8.0:
        return "B"
    elif floored >= 7.0:
        return "C"
    elif floored >= 6.0:
        return "D"
    else:
        return "F"
    
letter_grade(9.0)


'A'

In [8]:
# TODO: A2
def count_vowels(s: str) -> dict:
    vowels = 'aeiou'
    s_lower = s.lower()
    result = {v: 0 for v in vowels}
    for char in s_lower:
        if char in vowels:
            result[char] += 1
    return result
count_vowels("juntra")

{'a': 1, 'e': 0, 'i': 0, 'o': 0, 'u': 1}

In [10]:

# TODO: A3
def rotate_tuple(t: tuple) -> tuple:
    if len(t) == 1:
        return t
    return (t[-1],) + t[:-1]

rotate_tuple(("g", "h", "c", "d"))  # ('d', 'a', 'b', 'c')

('d', 'g', 'h', 'c')

In [2]:

# --- Tests for Part A ---
assert letter_grade(9.0) == "A"
assert letter_grade(6.0) == "D"
assert letter_grade(5.0) == "F"

'''
assert count_vowels("Aye") == {'a': 1, 'e': 1}
assert count_vowels("Beautiful day") == {'a': 2, 'e': 1, 'i': 1, 'o': 0, 'u': 2} or        sum(count_vowels("Beautiful day").values()) == 6  # lenient check

assert rotate_tuple((1,2,3)) == (3,1,2)
assert rotate_tuple(("a",)) == ("a",)
print("Part A tests passed (if no AssertionError).")

'''

'\nassert count_vowels("Aye") == {\'a\': 1, \'e\': 1}\nassert count_vowels("Beautiful day") == {\'a\': 2, \'e\': 1, \'i\': 1, \'o\': 0, \'u\': 2} or        sum(count_vowels("Beautiful day").values()) == 6  # lenient check\n\nassert rotate_tuple((1,2,3)) == (3,1,2)\nassert rotate_tuple(("a",)) == ("a",)\nprint("Part A tests passed (if no AssertionError).")\n\n'


---
## Part B — Gradebook (Files, Dicts, Methods)

You are given `data/scores.csv` with columns: `student_id,name,quiz1,quiz2,quiz3`.

### B1. `read_scores_csv(path)`
Read the CSV and return a list of dictionaries, one per student. Convert quiz scores to `int`.

### B2. `compute_averages(records)`
Given the list from B1, return a **new** list where each dict has added keys:
- `"avg"`: average of quizzes (float)
- `"grade"`: result of `letter_grade(avg)`

### B3. `write_report(records, out_path="report.txt")`
Write a simple text report with one line per student:
`S001 Aye -> avg=9.0 grade=A`

**Note:** Use `with open(...)` to handle files safely.


In [13]:

# TODO: B1
import csv

def read_scores_csv(path: str) -> list[dict]:
    records = []
    with open(path, newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # Convert quiz scores to int
            row['quiz1'] = int(row['quiz1'])
            row['quiz2'] = int(row['quiz2'])
            row['quiz3'] = int(row['quiz3'])
            records.append(row)
    return records

# Example usage:
# print(read_scores_csv("data/scores.csv"))


In [14]:
# TODO: B2
def compute_averages(records: list[dict]) -> list[dict]:
    new_records = []
    for r in records:
        avg = (r['quiz1'] + r['quiz2'] + r['quiz3']) / 3
        grade = letter_grade(avg)
        new_r = r.copy()
        new_r['avg'] = float(f"{avg:.1f}")
        new_r['grade'] = grade
        new_records.append(new_r)
    return new_records


In [15]:

# TODO: B3
from pathlib import Path

def write_report(records: list[dict], out_path: str = "report.txt") -> None:
    with open(out_path, "w", encoding="utf-8") as f:
        for r in records:
            line = f"{r['student_id']} {r['name']} -> avg={r['avg']} grade={r['grade']}\n"
            f.write(line)

In [ ]:

# --- Tests for Part B ---
recs = read_scores_csv("data/scores.csv")
assert isinstance(recs, list) and len(recs) >= 5
assert all(isinstance(r, dict) for r in recs)
assert set(recs[0].keys()) >= {"student_id","name","quiz1","quiz2","quiz3"}
assert all(isinstance(r["quiz1"], int) for r in recs)

recs2 = compute_averages(recs)
assert "avg" in recs2[0] and "grade" in recs2[0]

out_file = "report.txt"
write_report(recs2, out_file)
p = Path(out_file)
assert p.exists() and p.stat().st_size > 0
print("Part B tests passed (if no AssertionError).")



---
## Part C — Simple Log Analysis (Files, Dicts)

You are given `data/access.log` with lines: `username,action,timestamp`.

### C1. `parse_log(path)`
Return a list of tuples `(username, action, timestamp_str)`.

### C2. `action_counts(parsed)`
Return a dict `{action: count}` across all lines.

### C3. `most_active_user(parsed)`
Return a tuple `(username, count)` for the user with the most actions. Break ties by lexicographic username (smallest first).


In [19]:
# TODO: C1
def parse_log(path: str) -> list[tuple]:
    result = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) == 3:
                result.append(tuple(parts))
    return result

In [20]:
# TODO: C2
def action_counts(parsed: list[tuple]) -> dict:
    counts = {}
    for entry in parsed:
        action = entry[1]
        counts[action] = counts.get(action, 0) + 1
    return counts


In [22]:
# TODO: C3
def most_active_user(parsed: list[tuple]) -> tuple[str, int]:
    user_counts = {}
    for entry in parsed:
        user = entry[0]
        user_counts[user] = user_counts.get(user, 0) + 1
    max_count = max(user_counts.values())
    top_users = [u for u, c in user_counts.items() if c == max_count]
    top_user = min(top_users)
    return (top_user, max_count)

In [ ]:

# --- Tests for Part C ---
parsed = parse_log("data/access.log")
assert isinstance(parsed, list) and len(parsed) >= 5
assert all(isinstance(t, tuple) and len(t) == 3 for t in parsed)

counts = action_counts(parsed)
assert isinstance(counts, dict) and sum(counts.values()) == len(parsed)

user, cnt = most_active_user(parsed)
assert isinstance(user, str) and isinstance(cnt, int) and cnt >= 1
print("Part C tests passed (if no AssertionError).")



---
## Part D — Menu‑Driven CLI (Loops, Conditionals, Methods)

Write a loop that repeatedly shows this menu until the user chooses Quit:

```
1) Show top student(s) by average
2) Show action counts from access.log
3) Export a CSV of student_id,name,avg,grade to data/gradebook_out.csv
4) Quit
```

Implement as `main()` that returns `None`. For (1), if multiple students tie for top average, print them all.


In [24]:
# TODO: D — menu CLI
import csv

def main() -> None:
    scores_path = "data/scores.csv"
    log_path = "data/access.log"
    gradebook_out = "data/gradebook_out.csv"

    while True:
        print("\n1) Show top student(s) by average")
        print("2) Show action counts from access.log")
        print("3) Export a CSV of student_id,name,avg,grade to data/gradebook_out.csv")
        print("4) Quit")
        choice = input("Choose an option (1-4): ").strip()

        if choice == "1":
            records = compute_averages(read_scores_csv(scores_path))
            if not records:
                print("No student records found.")
                continue
            max_avg = max(r["avg"] for r in records)
            top_students = [r for r in records if r["avg"] == max_avg]
            print("Top student(s) by average:")
            for r in top_students:
                print(f"{r['student_id']} {r['name']} -> avg={r['avg']} grade={r['grade']}")
        elif choice == "2":
            parsed = parse_log(log_path)
            counts = action_counts(parsed)
            print("Action counts:")
            for action, count in counts.items():
                print(f"{action}: {count}")
        elif choice == "3":
            records = compute_averages(read_scores_csv(scores_path))
            with open(gradebook_out, "w", newline='', encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow(["student_id", "name", "avg", "grade"])
                for r in records:
                    writer.writerow([r["student_id"], r["name"], r["avg"], r["grade"]])
            print(f"Exported gradebook to {gradebook_out}")
        elif choice == "4":
            print("Goodbye!")
            break
        else:
            print("Invalid option. Please choose 1-4.")

# Optional: Uncomment below to run interactively in notebook
# main()

# --- IGNORE ---


In [ ]:
wite a function to takes an integer as an input scores
if the score is greater than or equal to 50, print "Pass"
if the score is greater than or equal to 75, print "Good"
if the score is exactly 100, print "Perfect"

In [18]:
def check_score(score: int):
    if score == 100:
        print("Perfect")
    elif score >= 75:
        print("Good")
    elif score >= 50:
        print("Pass")
check_score(85) 


Good


In [ ]:
write a program that returns the count of voetls in the string. Use a for loop to go through each character in the string.
    
Input ==> apple, Output ==> 2
Input ==> hello world, Output ==> 3
Input ==> balloon, Output ==> 3

In [28]:
def count_vowels_in_string(s: str) -> int:
    vowels = "aeiou"
    count = 0
    for char in s.lower():
        if char in vowels:
            count += 1
    return count



---
## Submission checklist
- [ ] All TODOs completed
- [ ] Notebook runs top-to-bottom without errors
- [ ] `report.txt` generated
- [ ] (Optional) `data/gradebook_out.csv` generated via menu option 3